In [30]:
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/05 13:23:15 WARN Utils: Your hostname, Ashrafs-iMac.local resolves to a loopback address: 127.0.0.1; using 172.23.23.25 instead (on interface en0)
23/03/05 13:23:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 13:23:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [45]:
df_fhvhv = spark.read \
.option("header",True) \
.schema(df_fhvhv_schema) \
.csv("fhvhv_tripdata_2021-06.csv.gz")

In [46]:
df_fhvhv.select("dispatching_base_num", "pickup_datetime", "Affiliated_base_number").show()

+--------------------+-------------------+----------------------+
|dispatching_base_num|    pickup_datetime|Affiliated_base_number|
+--------------------+-------------------+----------------------+
|              B02764|2021-06-01 00:02:41|                B02764|
|              B02764|2021-06-01 00:16:16|                B02764|
|              B02764|2021-06-01 00:27:01|                B02764|
|              B02764|2021-06-01 00:46:08|                B02764|
|              B02510|2021-06-01 00:45:42|                  null|
|              B02510|2021-06-01 00:18:15|                  null|
|              B02510|2021-06-01 00:33:06|                  null|
|              B02510|2021-06-01 00:46:27|                  null|
|              B02764|2021-06-01 00:48:06|                B02764|
|              B02875|2021-06-01 00:18:54|                B02875|
|              B02875|2021-06-01 00:31:02|                B02875|
|              B02875|2021-06-01 00:41:53|                B02875|
|         

In [47]:
df_fhvhv.count()

14961892

In [48]:
df_fhvhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [44]:
df_fhvhv_schema = types.StructType([
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropoff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("SR_Flag", types.StringType(), True),
    types.StructField("Affiliated_base_number", types.StringType(), True),
])

In [50]:
df_fhvhv \
.repartition(12) \
.write.parquet('data', mode= 'overwrite')

23/03/05 14:27:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/03/05 14:27:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [51]:
df_fhvhv_parquet = spark.read.parquet('data')

In [52]:
df_fhvhv_parquet.select('pickup_datetime').show()

+-------------------+
|    pickup_datetime|
+-------------------+
|2021-06-11 12:28:02|
|2021-06-13 08:46:22|
|2021-06-28 18:39:27|
|2021-06-05 18:15:05|
|2021-06-06 19:15:38|
|2021-06-07 18:59:58|
|2021-06-09 15:27:48|
|2021-06-27 03:39:56|
|2021-06-09 11:39:03|
|2021-06-20 11:16:26|
|2021-06-25 16:14:53|
|2021-06-13 18:13:03|
|2021-06-14 03:50:52|
|2021-06-25 18:37:30|
|2021-06-05 14:27:04|
|2021-06-28 21:40:22|
|2021-06-29 22:08:20|
|2021-06-25 14:41:35|
|2021-06-09 22:13:23|
|2021-06-30 22:01:58|
+-------------------+
only showing top 20 rows



In [53]:
df_fhvhv_parquet.createOrReplaceTempView('df_fhvhv_parquet')

In [54]:
spark.sql("""
SELECT 
    COUNT(1)
FROM
    df_fhvhv_parquet
WHERE pickup_datetime 
    BETWEEN "2021-06-15 00:00:00" AND "2021-06-15 23:59:59"
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [55]:
df_fhvhv_parquet \
.filter(F.col("pickup_datetime").between("2021-06-15 00:00:00", "2021-06-15 23:59:59")) \
.count()

452470

In [84]:
spark.sql("""
SELECT 
MAX(TIMESTAMPDIFF(SECOND, pickup_datetime, dropoff_datetime))/3600 AS difference
FROM
    df_fhvhv_parquet
""").show()

+----------------+
|      difference|
+----------------+
|66.8788888888889|
+----------------+



In [71]:
df_zones = spark.read \
.option("header", True) \
.schema(df_fhvhv_schema) \
.csv('zones')

In [65]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [72]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [69]:
df_fhvhv_schema = types.StructType([
    types.StructField("LocationID", types.IntegerType(), True),
    types.StructField("Borough", types.StringType(), True),
    types.StructField("Zone", types.StringType(), True),
    types.StructField("service_zone", types.StringType(), True),
])

In [73]:
df_zones.createOrReplaceTempView("df_zones")

In [77]:
spark.sql(
"""
SELECT
       df_zones.Zone, COUNT(PULocationID)
FROM 
    df_fhvhv_parquet
JOIN 
    df_zones
ON 
    df_fhvhv_parquet.PULocationID = df_zones.LocationID
    
GROUP BY
    1
ORDER BY
    2 DESC
"""

).show()

+--------------------+-------------------+
|                Zone|count(PULocationID)|
+--------------------+-------------------+
| Crown Heights North|             231279|
|        East Village|             221244|
|         JFK Airport|             188867|
|      Bushwick South|             187929|
|       East New York|             186780|
|TriBeCa/Civic Center|             164344|
|   LaGuardia Airport|             161596|
|            Union Sq|             158937|
|        West Village|             154698|
|             Astoria|             152493|
|     Lower East Side|             151020|
|        East Chelsea|             147673|
|Central Harlem North|             146402|
|Williamsburg (Nor...|             143683|
|          Park Slope|             143594|
|  Stuyvesant Heights|             141427|
|        Clinton East|             139611|
|West Chelsea/Huds...|             139431|
|             Bedford|             138428|
|         Murray Hill|             137879|
+----------